In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import transformers
import torch
import torchvision
import torchaudio
import numpy as np
import pandas as pd
import moviepy
import librosa
import glob
import moviepy.editor as mp
from torch.utils.data import TensorDataset, DataLoader, random_split
import librosa.display
import matplotlib.pyplot as plt
import tarfile
import matplotlib
import tqdm
import PIL
import cv2 as cv
import os
from os import walk, listdir
import shutil
from tqdm import tqdm
import pickle
from transformers import AutoTokenizer, BertForTokenClassification, BertForSequenceClassification,BertPreTrainedModel, BertModel, BertTokenizer, AutoModel
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from scipy.io.wavfile import read
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from transformers import ViTFeatureExtractor, ViTModel
import requests

EXTRACTING TEXT FEATURES FROM TRANSCRIPTIONS

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

In [ ]:
VIDEO_DIR = '/path/to/folder/containing/transcriptions'
TAREGT_DIR = '/path/to/folder/for/storing/features'
CATEGORY = "Explicit Hate Videos"

In [ ]:
allEmbedding ={}
combined_df = pd.read_csv(f'{VIDEO_DIR}/transcriptions_{CATEGORY}.csv')
df = pd.read_excel(f'{VIDEO_DIR}/{CATEGORY}.xlsx')
cnt = 1

for filename in (df['Video_ID']):
  try:
    file = filename + '.wav'
    #print(file)
    i = np.where(combined_df == file)[0][0]
    inputs = tokenizer(combined_df['Transcription'][i], return_tensors="pt", truncation = True, max_length=512, padding='max_length', add_special_tokens=True)
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        allEmbedding[i]= last_hidden_states.detach().numpy()
    del(outputs)
    print(cnt)
    cnt = cnt+1
  except:
    print(f"Error processing {file}")
    allEmbedding[i] = np.zeros((1,512,768), dtype=float)
    cnt = cnt+1
    pass

In [ ]:
arr1 = np.array(list(allEmbedding.values()))

In [ ]:
arr2 = arr1.reshape((500, 512, 768))

In [ ]:
np.save(f"{TARGET_DIR}/{CATEGORY}2DBERTembedding.npy",arr2)